In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import sqlalchemy
import sqlite3
from subprocess import check_output
import os

BASE1_path = '/kaggle/input/data-chap1-sql-intermediate/'

BASE2_path = '/kaggle/input/data-chap2-sql-intermediate/'

%sql sqlite://

'Connected: @None'

In [2]:
matches_2013_2014 = pd.read_table(os.path.join(BASE1_path, 'matches_2013_2014.txt'), sep = ';')
matches_2013_2014.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,1237,1,2013/2014,1,2014-03-29,8475,9989,2,0
1,1238,1,2013/2014,1,2014-03-29,9991,8573,0,1
2,1239,1,2013/2014,2,2014-04-05,9989,9991,1,0
3,1240,1,2013/2014,2,2014-04-05,8573,8475,0,0
4,1241,1,2013/2014,3,2014-04-12,9991,8475,2,1


In [3]:
team = pd.read_table(os.path.join(BASE2_path, 'team.txt'), sep = ';')
team.head()

,id,team_api_id,team_long_name,team_short_name
0,1,9987,KRC Genk,GEN
1,2,9993,Beerschot AC,BAC
2,3,10000,SV Zulte-Waregem,ZUL
3,4,9994,Sporting Lokeren,LOK
4,5,9984,KSV Cercle Brugge,CEB


In [4]:
match = pd.read_table(os.path.join(BASE1_path, 'match.txt'), sep = ';')
match.head()

,id,country_id,season,stage,date,hometeam_id,awayteam_id,home_goal,away_goal
0,757,1,2011/2012,1,2011-07-29,1773,8635,2,1
1,758,1,2011/2012,1,2011-07-30,9998,9985,1,1
2,759,1,2011/2012,1,2011-07-30,9987,9993,3,1
3,760,1,2011/2012,1,2011-07-30,9991,9984,0,1
4,761,1,2011/2012,1,2011-07-30,9994,10000,0,0


In [5]:
country = pd.read_table(os.path.join(BASE1_path, 'EU_country.txt'), sep = ';')
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [6]:
league = pd.read_table(os.path.join(BASE2_path, 'league.txt'), sep = ';')
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


## create table

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////football_EU_league', echo = False)

match.to_sql('match', con = engine)
matches_2013_2014.to_sql('matches_2013_2014', con = engine)
country.to_sql('country', con = engine)
team.to_sql('team', con = engine)
league.to_sql('league', con = engine)

# PRACTICEs
## 1. WHERE are the Subqueries?
### 1.1 Filtering using scalar subqueries
Subqueries are incredibly powerful for performing complex filters and transformations. You can filter data based on single, scalar values using a subquery in ways you cannot by using WHERE statements or joins. Subqueries can also be used for more advanced manipulation of your data set. You will likely encounter subqueries in any real-world setting that uses relational databases.

In this exercise, you will generate a list of matches where the total goals scored (for both teams in total) is more than 3 times the average for games in the matches_2013_2014 table, which includes all games played in the 2013/2014 season.

#### Instructions 
**Step 1.** Calculate triple the average home + away goals scored across all matches. This will become your subquery in the next step. 

Note that this column does not have an alias, so it will be called ?column? in your results.

In [8]:
pd.read_sql(
    """ 
        SELECT 3 * AVG(home_goal + away_goal)
        FROM matches_2013_2014;    
    """, con = engine)

,3 * AVG(home_goal + away_goal)
0,8.43


**Step 2.**
Select the `date`, `home goals`, and `away goals` in the main query.

Filter the main query for matches where the `total goals` scored `exceed the value in the subquery`.

In [9]:
pd.read_sql(
    """ 
        SELECT date, home_goal, away_goal
        FROM matches_2013_2014

        WHERE (home_goal + away_goal) > (SELECT 3 * AVG(home_goal + away_goal)
                                                FROM matches_2013_2014);     
    """, con = engine)

,date,home_goal,away_goal
0,2013-12-14,6,3
1,2014-03-22,6,3
2,2013-10-30,7,3


### 1.2. Filtering using a subquery with a list

Your goal in this exercise is to generate a list of teams that never played a game in their home city. Using a subquery, you will generate a list of unique `hometeam_ID` values from the unfiltered match table to exclude in the team table's `team_api_ID` column.

In addition to filtering using a single-value (scalar) subquery, you can create a list of values in a subquery to filter data based on a complex set of conditions. This type of subquery generates a one column reference list for the main query. As long as the values in your list match a column in your main query's table, you don't need to use a join -- even if the list is from a separate table.


#### Instructions

Create a subquery in the **`WHERE`** clause that retrieves all **unique** `hometeam_ID` values from the `match` table.

Select the `team_long_name` and `team_short_name` from the `team` table. Exclude all values from the subquery in the main query

In [10]:
pd.read_sql(
    """ 
        SELECT team_long_name, team_short_name
        FROM team 
        
        WHERE team_api_id NOT IN (SELECT DISTINCT hometeam_id 
                                  FROM match);    
    """, con = engine)

,team_long_name,team_short_name
0,FCV Dender EH,DEN
1,KSV Roeselare,ROS
2,Tubize,TUB
3,Royal Excel Mouscron,MOU
4,KAS Eupen,EUP
5,Middlesbrough,MID
6,Portsmouth,POR
7,Birmingham City,BIR
8,Blackpool,BLA
9,Bournemouth,BOU


So, there are 52 teams without any home games recorded in this database.

### Filtering with more complex subquery conditions

In the previous exercise, you generated a list of teams that have no home matches listed in the soccer database using a subquery in **WHERE**. Let's do some further exploration in this database by creating a list of teams that **scored 8 or more goals in a home match**.

In order to do this, you will construct a subquery in the **WHERE** statement with its own filtering condition.

#### Instructions

Create a subquery in **WHERE** clause that retrieves all `hometeam_ID` values from `match` with a `home_goal` score greater than or equal to 8.

Select the `team_long_name` and `team_short_name` from the `team` table. Include all values from the subquery in the main query.

In [11]:
pd.read_sql(
    """ 
        SELECT team_long_name, team_short_name
        FROM team
        WHERE team_api_id IN (SELECT DISTINCT hometeam_id
                               FROM match
                               WHERE home_goal >= 8);    
    """, con = engine)

,team_long_name,team_short_name
0,Manchester United,MUN
1,Chelsea,CHE
2,Southampton,SOU
3,FC Bayern Munich,BMU
4,Real Madrid CF,REA
5,FC Barcelona,BAR


## 2. Subqueries in FROM
### 2.1 Joining Subqueries in FROM

The match table in the `European Soccer Database` does not contain `country` or `team` names. You can get this information by joining it to the country table, and use this to aggregate information, such as the number of matches played in each country.

If you're interested in filtering data from one of these tables, you can also create a subquery from one of the tables, and then join it to an existing table in the database. A subquery in **FROM** is an effective way of answering detailed questions that requires filtering or transforming data before including it in your final results.

Your goal in this exercise is to generate a subquery using the match table, and then join that subquery to the country table to calculate information about matches with 10 or more goals in total!

#### Instructions 

**Step 1.** Create the subquery to be used in the next step, which selects the `country ID` and `match ID` (id) from the `match` table.

Filter the query for matches with greater than or equal to 10 goals.

In [12]:
pd.read_sql(
    """ 
        SELECT country_id, id 
        FROM match
        WHERE (home_goal + away_goal) >= 10;    
    """, con = engine)

,country_id,id
0,1729,3093
1,1729,3369
2,1729,3566
3,7809,9211
4,13274,14224
5,21518,23444
6,21518,24016
7,21518,24114
8,21518,24123


**Step 2.** Construct a subquery that selects only matches with 10 or more `total goals`.

**Inner join** the subquery onto country in the main query.

Select name from country and count the id column from `match`.

In [13]:
pd.read_sql(
    """ 
        SELECT c.name AS country_name,
               COUNT(sub.id) AS matches
        FROM country AS c
        INNER JOIN (SELECT country_id, id 
                    FROM match
                    WHERE (home_goal + away_goal) >= 10) AS sub
        ON c.id = sub.country_id
        GROUP BY country_name;    
    """, con = engine)

,country_name,matches
0,Netherlands,1
1,Spain,4
2,Germany,1
3,England,3


### 2.2. Building on Subqueries in FROM
In the previous exercise, you found that `England`, `Netherlands`, `Germany` and `Spain` were the only `countries` that had matches in the database where 10 or more goals were scored overall. Let's find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were `home` vs. `away goals`.

You'll notice that in this exercise, the table alias is excluded for every column selected in the `main query`. This is because the main query is extracting data from the subquery, which is treated as a `single table`.

#### Instructions

Complete the subquery inside the **FROM** clause. Select the `country name` from the `country` table, and the `home goal`, and `away goal` columns from the `match` table.

Create a column in the subquery that adds `home` and `away goals`, called `total_goals`. This will be used to filter the main query.

Select the `country, date, home goals`, and `away goals` in the main query.

Filter the main query for games with 10 or more `total goals`.

In [14]:
pd.read_sql(
    """ 
        SELECT country, date, home_goal, away_goal
        
        FROM (SELECT 
                    c.name AS country, 
                    m.date, m.home_goal, m.away_goal,
                   (m.home_goal + m.away_goal) AS total_goals
                FROM match AS m
                LEFT JOIN country AS c
                    ON m.country_id = c.id
             ) AS subquery
             
        WHERE total_goals >= 10;    
    """, con = engine)

,country,date,home_goal,away_goal
0,England,2011-08-28,8,2
1,England,2012-12-29,7,3
2,England,2013-05-19,5,5
3,Germany,2013-03-30,9,2
4,Netherlands,2011-11-06,6,4
5,Spain,2013-10-30,7,3
6,Spain,2015-04-05,9,1
7,Spain,2015-05-23,7,3
8,Spain,2014-09-20,2,8


## 3. Subqueries in SELECT

### 3.1 Add a subquery to the SELECT clause
Subqueries in **SELECT** statements generate a single value that allow you to pass an aggregate value down a data frame. This is useful for performing calculations on data within your database.

In the following exercise, you will construct a query that calculates the average number of goals per match in each country's league.

#### Instructions

In the subquery, select the average total goals by adding `home_goal` and `away_goal`.

Filter the results so that only the average of goals in the `2013/2014 season` is calculated.

In the main query, select the average total goals by adding `home_goal` and `away_goal`. This calculates the `average goals` for each `league`.

Filter the results in the main query the same way you filtered the subquery. **Group** the query **by** the `league name`.

In [15]:
pd.read_sql(
    """ 
        SELECT l.name AS league,
               ROUND(AVG(m.home_goal + m.away_goal), 2) AS avg_goals,
               (SELECT ROUND(AVG(home_goal + away_goal), 2) 
                       FROM match
                       WHERE season = '2013/2014') AS overall_avg
        FROM league AS l
        LEFT JOIN match AS m
            ON l.country_id = m.country_id

        WHERE season = '2013/2014'
        GROUP BY l.name;    
    """, con = engine)

,league,avg_goals,overall_avg
0,Switzerland Super League,2.89,2.77
1,Poland Ekstraklasa,2.64,2.77
2,Netherlands Eredivisie,3.20,2.77
3,Scotland Premier League,2.75,2.77
4,France Ligue 1,2.46,2.77
5,Spain LIGA BBVA,2.75,2.77
6,Germany 1. Bundesliga,3.16,2.77
7,Italy Serie A,2.72,2.77
8,Portugal Liga ZON Sagres,2.37,2.77
9,England Premier League,2.77,2.77


We can see that these following leagues **`EPL, Buldesliga, Netherlands Eredivisie`** and **`Switzerland Super League`** tended to score higher than average.

### 3.2 Subqueries in Select for Calculations
Subqueries in **SELECT** are a useful way to create calculated columns in a query. A subquery in **SELECT** can be treated as a single numeric value to use in your calculations. When writing queries in **SELECT**, it's important to remember that filtering the main query does not filter the subquery -- and vice versa.

In the previous exercise, you created a column to compare each league's average total goals to the overall average goals in the `2013/2014 season`. In this exercise, you will add a column that directly compares these values by **subtracting the overall average from the subquery**.

#### Instructions

Select the average goals scored in a match for each league in the main query.

Select the average goals scored in a match overall for the `2013/2014 season` in the subquery.

Subtract the subquery from the average number of goals calculated for each league.

Filter the main query so that only games from the `2013/2014 season` are included.

In [16]:
pd.read_sql(
    """ 
        SELECT l.name AS league,
            ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
            ROUND(AVG(m.home_goal + m.away_goal) - (SELECT AVG(home_goal + away_goal)
                                                    FROM match 
                                                    WHERE season = '2013/2014'),
                 2) AS diff
        FROM league AS l
        LEFT JOIN match AS m
            ON l.country_id = m.country_id

        WHERE season = '2013/2014'
        GROUP BY l.name;    
    """, con = engine)

,league,avg_goals,diff
0,Switzerland Super League,2.89,0.12
1,Poland Ekstraklasa,2.64,-0.13
2,Netherlands Eredivisie,3.20,0.43
3,Scotland Premier League,2.75,-0.02
4,France Ligue 1,2.46,-0.31
5,Spain LIGA BBVA,2.75,-0.02
6,Germany 1. Bundesliga,3.16,0.39
7,Italy Serie A,2.72,-0.04
8,Portugal Liga ZON Sagres,2.37,-0.40
9,England Premier League,2.77,0.00


Games in the `Netherlands` tend to score the **highest** number of goals on `average` in this season.

## 4. Subqueries everywhere!

### 4.1. ALL the Subqueries EVERYWHERE
In `soccer leagues`, games are played at different stages. Winning teams progress from one stage to the next, until they reach the `final stage`. In each stage, the stakes become higher than the previous one. The match table includes data about the different stages that each match took place in.

In this lesson, you will build a final query across 3 exercises that will contain three subqueries -- one in the **SELECT** clause, one in the **FROM** clause, and one in the **WHERE** clause. In the final exercise, your query will extract data examining the average goals scored in each stage of a `match`. 

Does the average number of goals scored change as the stakes get higher from one stage to the next?

#### Instructions

Extract the average number of home and away team goals in two **SELECT** `subqueries`.

Calculate the average home and away goals for the specific stage in the main query.

Filter both subqueries and the main query so that only data from the `2012/2013 season` is included.

**Group** the query **by** the `m.stage` column.

In [17]:
pd.read_sql(
    """ 
        SELECT m.stage,
               ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
               ROUND((SELECT AVG(home_goal + away_goal) 
                       FROM match 
                       WHERE season = '2012/2013'),2
                    ) AS overall
        FROM match AS m
        WHERE season = '2012/2013'
        GROUP BY m.stage;    
    """, con = engine)

,stage,avg_goals,overall
0,29,2.63,2.77
1,4,2.80,2.77
2,34,2.68,2.77
3,32,2.75,2.77
4,9,2.70,2.77
5,7,2.69,2.77
6,10,2.96,2.77
7,35,2.43,2.77
8,38,3.17,2.77
9,15,2.76,2.77


### 4.2. Add a subquery in FROM
In the previous exercise, you created a data set listing the `average home` and `away goals` in each match stage of the `2012/2013 match season`.

In this next step, you will turn the main query into a subquery to extract a list of stages where the `average home goals` in a stage is higher than the overall average for `home goals` in a match.

#### Instructions

Calculate the average home goals and average away goals from the match table for each stage in the **FROM** `clause subquery`.

Add a subquery to the **WHERE** clause that calculates the overall `average home goals`.

Filter the main query for stages where the average home goals is higher than the overall average.

Select the stage and `avg_goals` columns from the s subquery into the main query.

In [18]:
pd.read_sql(
    """ 
        SELECT s.stage,
                ROUND(s.avg_goals, 2) AS avg_goals
        FROM (SELECT stage,
                     AVG(home_goal + away_goal) AS avg_goals
                FROM match
                WHERE season = '2012/2013'
                GROUP BY stage) AS s
        WHERE s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                                FROM match 
                                WHERE season = '2012/2013');    
    """, con = engine)

,stage,avg_goals
0,4,2.80
1,10,2.96
2,38,3.17
3,6,2.78
4,12,3.23
5,36,2.90
6,31,3.06
7,30,2.87
8,21,2.90
9,3,2.83


Quite a few `stages` were eliminated with this filter.

### 4.3. Add a subquery in SELECT
In the previous exercise, you added a subquery to the **FROM** statement and selected the stages where the number of `average goals` in a stage exceeded the overall average number of goals in the `2012/2013 match` season. In this final step, you will add a subquery in **SELECT** to compare the average number of goals scored in each stage to the total.

#### Instructions
Create a subquery in **SELECT** that yields the average goals scored in the `2012/2013 season`. Name the new column `overall_avg`.

Create a subquery in **FROM** that calculates the average goals scored in each stage during the `2012/2013 season`.

Filter the main query for stages where the average goals exceeds the overall average in `2012/2013`.

In [19]:
pd.read_sql(
    """ 
        SELECT s.stage,
                ROUND(s.avg_goals, 2) AS avg_goal,
                (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
                    FROM (SELECT stage,
                                 AVG(home_goal + away_goal) AS avg_goals
                          FROM match
                          WHERE season = '2012/2013'
                          GROUP BY stage) AS s
        WHERE s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                             FROM match 
                             WHERE season = '2012/2013');    
    """, con = engine)

,stage,avg_goal,overall_avg
0,4,2.80,2.772699
1,10,2.96,2.772699
2,38,3.17,2.772699
3,6,2.78,2.772699
4,12,3.23,2.772699
5,36,2.90,2.772699
6,31,3.06,2.772699
7,30,2.87,2.772699
8,21,2.90,2.772699
9,3,2.83,2.772699
